# Digital Twin Experiment 👯‍♀️🔋

In [1]:
import pybamm
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import liionpack


In [2]:
pybamm.parameter_sets.Chen2020

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

In [3]:
pybamm.PARAMETER_PATH


['/Users/eirikbaekkelund/opt/anaconda3/lib/python3.9/site-packages',
 '/Users/eirikbaekkelund/Desktop/RePack/Python',
 '/Users/eirikbaekkelund/opt/anaconda3/lib/python3.9/site-packages/pybamm/input/parameters']

In [4]:
# creates copy folder of all parameters 
%%bash
pybamm_edit_parameter lithium_ion

In [6]:
# creates copy of negative electrode parameters and csv file that can be made for tuning the params
%%bash 
cp -r lithium_ion/negative_electrodes/graphite_Chen2020/. lithium_ion/negative_electrodes/test_parameters

In [7]:
# we need to tune / have all parameters to define a chemistry, so we can start from a default set
chemistry = pybamm.parameter_sets.Chen2020

In [8]:
# dict with parameter where keys are params and values are read from csv files
chemistry

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

In [9]:
# we can change chemistry by setting to user defined: name.csv - here i named it test_parameters
chemistry['negative electrode'] = 'test_parameters'

In [10]:
chemistry

{'chemistry': 'lithium_ion',
 'cell': 'LGM50_Chen2020',
 'negative electrode': 'test_parameters',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

In [12]:
parameter_values = pybamm.ParameterValues(chemistry = chemistry)

In [13]:
# I changed "Negative electrode charge transfer coefficient" from 0.5 to 0.3
parameter_values['Negative electrode charge transfer coefficient']

0.3

In [22]:
# take the new parameter set to pybamm for others to use by sending it into lithium_ion --> negative_electrodes
%%bash
pybamm_add_parameter lithium_ion/negative_electrodes/test_parameters lithium_ion negative_electrodes

SyntaxError: invalid syntax (3025649900.py, line 3)

In [ ]:
%%bash 
